In [ ]:
# !pip install git+git://github.com/adamnpeace/nlpt5.git
# !pip install --user nltk pandas numpy gdown tqdm
# !python -m nltk.downloader punkt

In [1]:
import os, pickle, time, random, logging, json, gc
from datetime import datetime
from tqdm import tqdm
import gdown
import numpy as np
import pandas as pd
from nlpt5 import pipeline
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/dojo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
# Download T5 model from GDrive
gdown.download('https://drive.google.com/uc?id=1-1YIOyMdCvdhO9Z_DRHRBk20DhLB9XSp',
               quiet=True,
               output='models/final_T5_model.zip')
!unzip models/final_T5_model.zip -d models
!mv models/t5-small-e2e-qg-7k models/final_T5_model
!rm models/final_T5_model.zip

In [2]:
T5_PATH = 'models/final_T5_model'

In [3]:
def get_t5_model(path):
    return pipeline('e2e-qg', model=path)

In [4]:
def t5_inference(t5_model, text):
    prediction = t5_model(text)
    prediction = prediction[0] if len(prediction) > 0 else ''
    return prediction

## T5 Model Inference

In [6]:
t5_model = get_t5_model(T5_PATH)

In [7]:
data_test = json.load(open('data/data_test.json', 'r'))

In [8]:
outputs = []
for r in data_test:
    outputs.append(t5_inference(t5_model, r['passages']))

In [9]:
outputs

['Who wrote the book "The Crown"?',
 'When did Lopez release her second studio album, "The Wedding Planner"?',
 'In what year did Nicholas Alexei die?',
 'Who wrote "Up Where We Belong" for the 1982 film An Officer and a Gentleman"?',
 'When was the Balfour Declaration issued?',
 'In what year was the Historia ecclesiastica gentis Anglorum completed?',
 'Tony Blair is a funded adviser to the Mubadala Development Company in what city?',
 'Who played the fictional character on the television show Seinfeld?',
 'In 1997, Sharif authorised nuclear testings in which country?',
 'When did Thurgood Marshall serve as a Justice of the Supreme Court of the United States?',
 '',
 "Who was the spokesman for the State Department's ambitious plans to rebuild Europe?",
 '"I\'m Walkin" and "A Teenager\'s Romance" were both released in what month of 1957?',
 "What is the name of the international women's tennis tournament in this country?",
 'In 1988 she penned The Witching Hour as an expression of her 

In [10]:
pd.DataFrame({'Truths (T5)': [r['clues'] for r in data_test], 'Predicted (T5)': outputs}).to_csv('data/results_T5.csv', index=False)